# Demo Load from Youtube and forward to Graph Universe

In [ ]:
# imports
from pytube import YouTube, Playlist, Channel
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_community.document_loaders import YoutubeLoader


In [ ]:
# load playlist
playlist = Playlist("https://www.youtube.com/watch?v=GUh7QyCwxGk&list=PL6RpkC85SLQCGjMBsoQYlMrLmbZaEWM6U")
print(playlist)
playlist.owner


In [ ]:
# loop playlist
max = 2
cur = 0
for video_url in playlist:
    cur+=1
    if cur > max:
        break

    print(video_url)
    
    yt = YouTube(video_url)
    print(yt.thumbnail_url)
    print(yt.metadata)
    
    loader = YoutubeLoader.from_youtube_url(
                    youtube_url=video_url, 
                    add_video_info=False,
                    language=["en", "de"],
                    translation="en",                    
                    #transcript_format=TranscriptFormat.CHUNKS,
                    #chunk_size_seconds=30, 
                )
    documents = loader.load()
    print(documents)       
